In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# plotting 환경 설정
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
plt.rcParams['figure.figsize'] = (5,5) # 그래프 크기
plt.rcParams['font.family'] = 'Malgun Gothic' # 글꼴
plt.rcParams['font.size'] = 12 # 글꼴 크기
plt.rcParams['axes.unicode_minus'] = False # 마이너스 표기

# 그래프가 노트북 안에 보이게 하기 위해
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 데이터를 구분하는 문자를 지정하는 옵션
vod8 = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/data/데이터 3차/데이터스쿨_3차_VOD_2308.csv', sep='\t', encoding='cp949')
vod9 = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/data/데이터 3차/데이터스쿨_3차_VOD_2309.csv', sep='\t', encoding='cp949')
# channel8 = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/data/데이터 3차/데이터스쿨3차_채널PGM_2308월.csv', sep='\t', encoding='cp949')
# channel9 = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/data/데이터 3차/데이터스쿨3차_채널PGM_2309월.csv', encoding='cp949')
con8 = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/data/데이터 3차/데이터스쿨_3차_콘텐츠_2308.csv', sep='\t', encoding='cp949')
con9 = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/data/데이터 3차/데이터스쿨_3차_콘텐츠_2309.csv', sep='\t', encoding='cp949')

In [5]:
v8 = vod8.copy()
v9 = vod9.copy()
c8 = con8.copy()
c9 = con9.copy()

In [10]:
con9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14387 entries, 0 to 14386
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subsr           14387 non-null  int64 
 1   series_nm       6458 non-null   object
 2   super_asset_nm  14387 non-null  object
 3   ct_cl           14387 non-null  object
 4   genre_of_ct_cl  14387 non-null  object
 5   SMRY            14387 non-null  object
 6   ACTR_DISP       14387 non-null  object
 7   disp_rtm        13260 non-null  object
 8   log_dt          14387 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1011.7+ KB


In [92]:
# con 데이터프레임을 header가 같은 열을 기준으로 합치기
con89 = pd.concat([con8, con9], ignore_index=True)
con89.tail(1)

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt
30063,64551000,NaN,슈퍼 토끼,키즈,학습,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내...",-,0:12,20230930121654


In [93]:
# 정규표현식을 사용하여 괄호 제거
import re

# series_nm 괄호와 괄호 안의 데이터 제거
con89['series_nm'] = con89['series_nm'].str.replace(r'\([^()]*\)', '', regex=True)
con89['series_nm'] = con89['series_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
con89['series_nm'] = con89['series_nm'].str.replace(r'\.\.\.', '', regex=True)
con89['series_nm'] = con89['series_nm'].str.replace(r'\(.*$', '', regex=True)
con89['series_nm'] = con89['series_nm'].str.replace('-', '')

# series_nm 괄호와 괄호 안의 데이터 제거
con89['super_asset_nm'] = con89['super_asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
con89['super_asset_nm'] = con89['super_asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
con89['super_asset_nm'] = con89['super_asset_nm'].str.replace(r'\.\.\.', '', regex=True)
con89['super_asset_nm'] = con89['super_asset_nm'].str.replace(r'\(.*$', '', regex=True)
con89['super_asset_nm'] = con89['super_asset_nm'].str.replace('-', '')

# n회 or n화 제거
con89['super_asset_nm'] = con89['super_asset_nm'].str.replace(r'\s\d+회', '', regex=True)


# 프로그램명 제일 마지막에 . 이 찍힌 경우가 있으니 . 제거
# con89['series_nm'] = con89['series_nm'].str.replace('.', '')
# con89['super_asset_nm'] = con89['super_asset_nm'].str.replace('.', '')

# 'series_nm'이 null인 경우, 'super_asset_nm'의 값으로 대체
con89['series_nm'].fillna(con89['super_asset_nm'], inplace=True)

con89.head(1)

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt
0,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,0:01,20230813071155


In [94]:
con89[con89["series_nm"].isnull()]

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt


In [95]:
# con 데이터프레임을 header가 같은 열을 기준으로 합치기
vod89 = pd.concat([vod8, vod9], ignore_index=True)
vod89.tail(1)

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
10656,60169000,놀자!,키즈,학습,407,[인기작가 박정섭의 그림책] 놀고 싶은 어린이들 모두 모두 모여라! 온몸으로 뛰어놀...,-,0:07,20230914210135


In [96]:
# asset_nm 괄호와 괄호 안의 데이터 제거
vod89['asset_nm'] = vod89['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
vod89['asset_nm'] = vod89['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
vod89['asset_nm'] = vod89['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
vod89['asset_nm'] = vod89['asset_nm'].str.replace(r'\(.*$', '', regex=True)
vod89['asset_nm'] = vod89['asset_nm'].str.replace('-', '')

# n회 or n화 제거
vod89['asset_nm'] = vod89['asset_nm'].str.replace(r'\s\d+회', '', regex=True)

vod89.head(1)

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,그것이알고싶다,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230812163507


In [97]:
# 새로운 변수 추가
new_column_name  = 'summary'
con89["summary"] = pd.Series(dtype='object')

new_column_name  = 'director'
con89["director"] = pd.Series(dtype='object')

new_column_name  = 'category'
con89["category"] = pd.Series(dtype='object')
con89.head(1)

,subsr,series_nm,super_asset_nm,ct_cl,genre_of_ct_cl,SMRY,ACTR_DISP,disp_rtm,log_dt,summary,director,category
0,65968000,핑크퐁 동물 동요,핑크퐁 동물 동요,키즈,기타,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ...",핑크퐁,0:01,20230813071155,NaN,NaN,NaN


In [98]:
new_column_name  = 'summary'
vod89["summary"] = pd.Series(dtype='object')

new_column_name  = 'director'
vod89["director"] = pd.Series(dtype='object')

new_column_name  = 'category'
vod89["category"] = pd.Series(dtype='object')
vod89.head(1)

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt,summary,director,category
0,65941000,그것이알고싶다,TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230812163507,NaN,NaN,NaN


In [99]:
new_order = ["series_nm", "summary", "ACTR_DISP", "director", "ct_cl", "genre_of_ct_cl", "category", "disp_rtm", "SMRY"]
con = con89[new_order]
con.tail(1)

,series_nm,summary,ACTR_DISP,director,ct_cl,genre_of_ct_cl,category,disp_rtm,SMRY
30063,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,0:12,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."


In [100]:
# 순서 변경
new_order = ["asset_nm", "summary", "ACTR_DISP", "director", "ct_cl", "genre_of_ct_cl", "category", "disp_rtm","SMRY"]
vod = vod89[new_order]
vod.tail(1)

,asset_nm,summary,ACTR_DISP,director,ct_cl,genre_of_ct_cl,category,disp_rtm,SMRY
10656,놀자!,NaN,-,NaN,키즈,학습,NaN,0:07,[인기작가 박정섭의 그림책] 놀고 싶은 어린이들 모두 모두 모여라! 온몸으로 뛰어놀...


In [101]:
# 이름 변경
con.columns = ['name', 'Summary', 'actors', 'director', 'BigCategory', 'SmallCategory', 'Category', 'runningtime', 'SMRY']
con.tail(1)

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
30063,슈퍼 토끼,NaN,-,NaN,키즈,학습,NaN,0:12,"[교보문고,YES24,알라딘 베스트셀러] 경주에 진 그 토끼는 어떻게 되었을까? 내..."


In [102]:
vod.columns = ['name', 'Summary', 'actors', 'director', 'BigCategory', 'SmallCategory', 'Category', 'runningtime', 'SMRY']
vod.tail(1)

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
10656,놀자!,NaN,-,NaN,키즈,학습,NaN,0:07,[인기작가 박정섭의 그림책] 놀고 싶은 어린이들 모두 모두 모여라! 온몸으로 뛰어놀...


In [103]:
vod_data = pd.concat([con, vod], ignore_index=True)
vodd = vod_data.copy()
vod_data.head(5)

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
0,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,NaN,0:01,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
1,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,NaN,0:01,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
2,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,NaN,0:02,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
3,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,NaN,0:02,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
4,지니강이 플러스 시즌7.,NaN,"헤이지니,럭키강이",NaN,키즈,기타,NaN,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...


In [104]:
# 'name'과 'BigCategory'를 기준으로 중복된 값 삭제
df_no_duplicates = vod_data.drop_duplicates(subset=['name', 'BigCategory'])
# vod_data = vod_data.reset_index(drop=True)
vod_data.tail(1)

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
40720,놀자!,NaN,-,NaN,키즈,학습,NaN,0:07,[인기작가 박정섭의 그림책] 놀고 싶은 어린이들 모두 모두 모여라! 온몸으로 뛰어놀...


In [105]:
# Category 구분
vod_data['Category'] = np.where(vod_data['BigCategory'] == '영화', '영화', np.where(vod_data['BigCategory'] == '키즈', '키즈', 'TV프로그램'))
vod_data

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY
0,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,키즈,0:01,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
1,핑크퐁 동물 동요,NaN,핑크퐁,NaN,키즈,기타,키즈,0:01,"겨울잠. 숲, 바다, 극지방 등 세계 여러 곳에 사는 동물 친구들을 만나러 가자! ..."
2,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,키즈,0:02,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
3,라바 키즈동요 Part4,NaN,"엘로우,레드",NaN,키즈,기타,키즈,0:02,징글벨 크리스마스. 안녕하신가 친구들! 메리 크리스마스~ 징글벨 크리스마스 징글벨 ...
4,지니강이 플러스 시즌7.,NaN,"헤이지니,럭키강이",NaN,키즈,기타,키즈,0:11,로봇 세상에서 킥보드를 타요. 로보카 폴리 브룸스 타운에서 킥보드를 타요~ / 새로...
...,...,...,...,...,...,...,...,...,...
40716,간질간질,NaN,-,NaN,키즈,학습,키즈,0:06,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...
40717,알사탕,NaN,-,NaN,키즈,학습,키즈,0:08,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고..."
40718,알사탕,NaN,-,NaN,키즈,학습,키즈,0:08,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고..."
40719,월간 아기상어,NaN,아기상어,NaN,키즈,학습,키즈,0:01,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...


### 확인

In [106]:
vod_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40721 entries, 0 to 40720
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           40721 non-null  object
 1   Summary        0 non-null      object
 2   actors         40717 non-null  object
 3   director       0 non-null      object
 4   BigCategory    40721 non-null  object
 5   SmallCategory  40721 non-null  object
 6   Category       40721 non-null  object
 7   runningtime    38429 non-null  object
 8   SMRY           40717 non-null  object
dtypes: object(9)
memory usage: 2.8+ MB


In [107]:
# '-', '[]'가 들어간 값 확인
filtered_rows = vod_data[vod_data['name'].str.contains('\-')]
# filtered_rows = vod_data[vod_data['name'].str.contains('\[\]')]
filtered_rows

,name,Summary,actors,director,BigCategory,SmallCategory,Category,runningtime,SMRY


In [108]:
vod_data.to_csv('vod_data.csv', index=False, encoding='cp949')